#問題3  
IWSLT15(en-vi)のデータセットに対してより高い精度(BLEU)を実現するプログラム(PyTorch)を作成せよ。ただし、プログラムは第13回の講義資料のプログラム(もしくはLab Work (6)で作成したプログラム)を改良して作成せよ。

In [ ]:
import requests
import torch
import torch.nn.functional as F
import torchtext

In [ ]:
#ライブラリのインポート、データのダウンロードを行います。
url = 'https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/'
train_en =[line.split() for line in requests.get(url+'train.en').text.splitlines()]
train_vi =[line.split() for line in requests.get(url+'train.vi').text.splitlines()]
test_en =[line.split() for line in requests.get(url+'tst2013.en').text.splitlines()]
test_vi =[line.split() for line in requests.get(url+'tst2013.vi').text.splitlines()]

In [ ]:
for i in range(10):
    print(train_en[i])
    print(train_vi[i])
print('# of line',len(train_en),len(train_vi),len(test_en),len(test_vi))

['Rachel', 'Pike', ':', 'The', 'science', 'behind', 'a', 'climate', 'headline']
['Khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu']
['In', '4', 'minutes', ',', 'atmospheric', 'chemist', 'Rachel', 'Pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.']
['Trong', '4', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'Rachel', 'Pike', 'giới', 'thiệu', 'sơ', 'lược', 'về', 'những', 'nỗ', 'lực', 'khoa', 'học', 'miệt', 'mài', 'đằng', 'sau', 'những', 'tiêu', 'đề', 'táo', 'bạo', 'về', 'biến', 'đổi', 'khí', 'hậu', ',', 'cùng', 'với', 'đoàn', 'nghiên', 'cứu', 'của', 'mình', '--', 'hàng', 'ngàn', 'người', 'đã', 'cống', 'hiến', 'cho', 'dự', 'án', 'này', '--', 'một

#1. Data preparation





### Make vocab

In [ ]:
def make_vocab(train_data, min_freg):
    vocab = {}
    for tokenlist in train_data:
        for token in tokenlist:
          #辞書を使ってトークンの出現回数をカウント
            if token not in vocab:
                vocab[token] = 0
            vocab[token] += 1
    #語彙リストの0〜3番⽬を<unk>,<pad>, <cls>, <eos>で予約
    vocablist = [('<unk>',0), ('<pad>,0'),('<cls>',0),('<eos>',0)]
    vocabidx = {}
    for token, freg in vocab.items():
        if freg >= min_freg:
            idx = len(vocablist)
            #min_freq以上の出現回数のトークンだけ語彙リスト(vocablist)と語彙インデックス(vocabidx)に登録
            vocablist.append((token,freg))
            vocabidx[token]=idx
    #<unk>,<pad>,<cls>,<eos>のインデックス登録
    vocabidx['<unk>'] =0
    vocabidx['<pad>'] =0
    vocabidx['<cls>'] =0
    vocabidx['<eos>'] =0
    return vocablist, vocabidx

In [ ]:
#今回はmin_freqを3に設定
vocablist_en,vocabidx_en = make_vocab(train_en,3)
vocablist_vi,vocabidx_vi = make_vocab(train_vi,3)
print('vocab size en:',len(vocablist_en))
print('vocab size vi:',len(vocablist_vi))

vocab size en: 24420
vocab size vi: 10666


### Pre-proces


In [ ]:
def preprocess(data, vocabidx):
    rr = []
    for tokenlist in data:
      #テキストの先頭に<cls>トークンを追加
        tkl = ['<cls>']
        for token in tokenlist:
          #トークンの追加。語彙リストに無いトークンは<unk>に変換
            tkl.append(token if  token in vocabidx else '<unk>')
          #テキストの末尾に<eos>トークンを追加
        tkl.append('<eos>')
        rr.append(tkl)
    return rr
train_en_prep = preprocess(train_en, vocabidx_en)
train_vi_prep = preprocess(train_vi, vocabidx_vi)
test_en_prep = preprocess(test_en, vocabidx_en)
#この部分はあっても無くても良い
for i in range(5):
    print(train_en_prep[i])
    print(train_vi_prep[i])
    print(test_en_prep[i])

['<cls>', 'Rachel', 'Pike', ':', 'The', 'science', 'behind', 'a', 'climate', 'headline', '<eos>']
['<cls>', 'Khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu', '<eos>']
['<cls>', 'When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', '<unk>', '.', '&quot;', '<eos>']
['<cls>', 'In', '4', 'minutes', ',', 'atmospheric', 'chemist', 'Rachel', 'Pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.', '<eos>']
['<cls>', 'Trong', '4', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'Rachel', 'Pike', 'giới', 'thiệu

In [ ]:
#訓練データのzip化(前処理済en, 前処理済vi)
train_data = list(zip(train_en_prep, train_vi_prep))
#訓練データのソーティング
train_data.sort(key = lambda x: (len(x[0]), len(x[1])))
#テストデータのZip化(前処理済en, en, vi)
test_data = list(zip(test_en_prep, test_en, test_vi))
#この部分はあっても無くても良い
for i in range(5):
    print(train_data[i])
    print(test_data[i])

(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', 'When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', '<unk>', '.', '&quot;', '<eos>'], ['When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', 'Envy', '.', '&quot;'], ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'Tôi', 'nghĩ', 'rằng', 'BắcTriều', 'Tiên', 'là', 'đất', 'nước', 'tốt', 'nhất', 'trên', 'thế', 'giới', 'và', 'tôi', 'thường', 'hát', 'bài', '&quot;', 'Chúng', 'ta', 'chẳng', 'có', 'gì', 'phải', 'ghen', 'tị', '.', '&quot;'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', 'And', 'I', 'was', 'very', 'proud', '.', '<eos>'], ['And', 'I', 'was', 'very', 'proud', '.'], ['Tôi', 'đã', 'rất', 'tự', 'hào', 'về', 'đất', 'nước', 'tôi', '.'])
(['<cls>

### Make batch

In [ ]:
def make_batch(data, batchsize=64):
    bb = []
    ben = []
    bvi = []
    for en,vi in data:
      #英語⽂だけbenにまとめ、ベトナム語⽂だけbviにまとめる
        ben.append(en)
        bvi.append(vi)
        if len(ben) >= batchsize:
          #バッチサイズと同じ⼤きさになったら、たまったバッチデータをbbに追加
            bb.append((ben, bvi))
            ben = []
            bvi = []
    if len(ben) > 0:
      #残ったbenと bviを忘れずにbbに追加
        bb.append((ben,bvi))
    return bb
train_data = make_batch(train_data, 64)
for i in range(5):
    print(train_data[i])

([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']

### Padding batch

In [ ]:
def padding_batch(b):
  #ミニバッチ内で⼀番⻑いトークン列の⻑さを得る
    maxlen=max([len(x) for x in b])
    for tkl in b:
        for i in range(maxlen-len(tkl)):
          #最⼤⻑と同じ⻑さになるように<pad>トークンの追加を繰り返す
            tkl.append('<pad>')

def padding(bb):
    for ben,bvi in bb:
      #英語バッチとベトナム語バッチと両⽅にパディングをする
        padding_batch(ben)
        padding_batch(bvi)
padding(train_data)
for i in range(3):
    print(train_data[i])

([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']

### Encoding

In [ ]:
#語彙インデックスを使ってトークンをID化
train_data = [([[vocabidx_en[token] for token in tokenlist] for tokenlist in ben],
               [[vocabidx_vi[token] for token in tokenlist] for tokenlist in bvi])for ben,bvi in train_data]
test_data = [([vocabidx_en[token] for token in enprep],en, vi) for enprep, en, vi in test_data]
for i in range(3):
    print(train_data[i])
    print(test_data[i])

([[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 

# LSTM model

In [ ]:
class LSTMEncDec(torch.nn.Module):
    def __init__(self,vocablist_x,vocabidx_x,vocablist_y,vocabidx_y):
        super(LSTMEncDec,self).__init__()
        self.dropout = torch.nn.Dropout(0.5)
        #encemb: エンコーダーの畳み込み層(300次元)
        self.encemb = torch.nn.Embedding(len(vocablist_x),512, padding_idx=vocabidx_x['<pad>'])
        #encrnn: エンコーダーのRNN計算ユニット(512✕512のFC層)
        self.encrnn = torch.nn.LSTM(512,512,3,dropout = 0.5,  bidirectional=True)
        #decemb: デコーダーの畳み込み層(512次元)
        self.decemb = torch.nn.Embedding(len(vocablist_y),512, padding_idx=vocabidx_y['<pad>'])
        #decrnn: デコーダーのRNN計算ユニット(512✕512のFC層)
        self.decrnn = torch.nn.LSTM(512,512,3,dropout = 0.5, bidirectional=True)
        #decout: デコーダーの出力(1024✕目的言語の語彙数)
        self.decout = torch.nn.Linear(1024,len(vocablist_y))

    def forward(self,x):
        #encoder
        #x: input (⽂⻑✕バッチサイズ)
        #y:output (⽂⻑✕バッチサイズ)
        x,y = x[0],x[1]
        e_x = self.encemb(x)
        n_x = e_x.size()[0]
        h = torch.zeros((6,x.size(1),512), dtype=torch.float32).to(DEVICE)
        c = torch.zeros((6,x.size(1),512), dtype=torch.float32).to(DEVICE)
        for i in range(n_x):
            output,(h,c)=self.encrnn(torch.unsqueeze(e_x[i],0),(h,c)) 
        output = torch.squeeze(output)
        #decoder
        e_y = self.decemb(y)
        #n_y=⽂⻑(J)+2 (<cls>と <eos>)
        n_y = e_y.size()[0]
        loss = torch.tensor(0.,dtype=torch.float32).to(DEVICE)
        for i in range(n_y-1):
            out,(h,c) = self.decrnn(torch.unsqueeze(e_y[i],0),(h,c))
            out = torch.squeeze(out)
            loss += F.cross_entropy(self.decout(out), y[i+1])
        return loss

    def evaluate(self,x,vocablist_y,vocabidx_y):
        # encoder
        #推論は1⽂ずつ⾏うので、 xには⽂⻑✕バッチサイズ1のミニバッチが⼊っている。
        e_x = self.encemb(x)
        n_x = e_x.size()[0]
        #エンコーダー部はforwardとほぼ同じ
        h = torch.zeros((6,x.size(1),512), dtype=torch.float32).to(DEVICE)
        c = torch.zeros((6,x.size(1),512), dtype=torch.float32).to(DEVICE)
        for i in range(n_x):
            output,(h,c)=self.encrnn(torch.unsqueeze(e_x[i],0),(h,c))
        # decoder
        #デコーダーの⼊⼒(バッチサイズ1)を作る。最初は<cls>トークンを⼊⼒する
        y = torch.tensor([vocabidx_y['<cls>']]).to(DEVICE)
        e_y = self.decemb(y)
        pred = []
        for i in range(50):
            out,(h,c) = self.decrnn(torch.unsqueeze(e_y,0),(h,c))
            out = out.view(1,1024)
            pred_id = self.decout(out).squeeze().argmax()
            #pred_idが予測する出⼒単語ID pred_idが<eos>のIDと等しければ推論終了
            if pred_id == vocabidx_y['<eos>']:
                break
            pred_y = vocablist_y[pred_id][0]
            pred.append(pred_y)
            #デコーダーは1単語ずつ処理をし、得られた出⼒を次の⼊⼒とする
            y[0] = pred_id
            e_y = self.decemb(y)
        return pred

# Train model

In [ ]:
list_train = []
list_loss = []

In [ ]:
def train():
    model = LSTMEncDec(vocablist_en,vocabidx_en,vocablist_vi,vocabidx_vi).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = LR)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
    for epoch in range(EPOCH):
        loss = 0
        step = 0
        for ben, bvi in train_data:
            ben = torch.tensor(ben,dtype=torch.int64).transpose(0,1).to(DEVICE)
            bvi = torch.tensor(bvi,dtype=torch.int64).transpose(0,1).to(DEVICE)
            optimizer.zero_grad()
            batchloss =model((ben,bvi))
            batchloss.backward()
            optimizer.step()
            loss = loss + batchloss.item()
            if step %100 == 0:
                print('step:',step,'batchloss:',batchloss.item())
            step+=1
        print('epoch',epoch,': loss',loss)
        list_loss.append(loss)
        if (epoch%2==0):
            torch.save(model.state_dict(),MODELNAME)
            list_train.append(test_during_train(epoch))
        scheduler.step()
    torch.save(model.state_dict(),MODELNAME)

In [ ]:
def test_during_train(epoch):
    total = 0
    correct = 0
    model = LSTMEncDec(vocablist_en,vocabidx_en,vocablist_vi,vocabidx_vi).to(DEVICE)
    model.load_state_dict(torch.load(MODELNAME))
    model.eval()
    ref = []
    pred =[]
    for enprep, en, vi in test_data:
        input =  torch.tensor([enprep], dtype=torch.int64).transpose(0,1).to(DEVICE)
        p = model.evaluate(input, vocablist_vi,vocabidx_vi)
        ref.append([vi])
        pred.append(p)
    bleu = torchtext.data.metrics.bleu_score(pred,ref)
    print('bleu of epoch {}: '.format(epoch),bleu)
    return bleu

In [ ]:
EPOCH = 20
LR = 0.001
MODELNAME ='iwslt15-en-vi-lstm-lr-0.001.model'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import time
start = time.time()
train()
print("Completed:", time.time()-start)

step: 0 batchloss: 9.263731002807617
step: 100 batchloss: 34.82604217529297
step: 200 batchloss: 50.13052749633789
step: 300 batchloss: 58.168724060058594
step: 400 batchloss: 62.2761344909668
step: 500 batchloss: 66.78204345703125
step: 600 batchloss: 72.72357940673828
step: 700 batchloss: 76.90071868896484
step: 800 batchloss: 97.08255004882812
step: 900 batchloss: 65.56108093261719
step: 1000 batchloss: 82.44290924072266
step: 1100 batchloss: 94.89924621582031
step: 1200 batchloss: 91.28659057617188
step: 1300 batchloss: 156.03683471679688
step: 1400 batchloss: 126.46376037597656
step: 1500 batchloss: 102.3239517211914
step: 1600 batchloss: 136.37159729003906
step: 1700 batchloss: 156.05300903320312
step: 1800 batchloss: 171.08604431152344
step: 1900 batchloss: 194.6428680419922
step: 2000 batchloss: 215.72842407226562
epoch 0 : loss 229030.49259853363
bleu of epoch 0:  0.01984742283821106
step: 0 batchloss: 2.905015468597412
step: 100 batchloss: 22.30035972595215
step: 200 batchlos

# Test model

In [ ]:
def test():
    total = 0
    correct = 0
    model = LSTMEncDec(vocablist_en,vocabidx_en,vocablist_vi,vocabidx_vi).to(DEVICE)
    model.load_state_dict(torch.load(MODELNAME))
    model.eval()
    ref = []
    pred =[]
    for enprep, en, vi in test_data:
        input =  torch.tensor([enprep], dtype=torch.int64).transpose(0,1).to(DEVICE)
        p = model.evaluate(input, vocablist_vi,vocabidx_vi)
        print('INPUT',en)
        print('REF',vi)
        print('MT',p)
        ref.append([vi])
        pred.append(p)
    bleu = torchtext.data.metrics.bleu_score(pred,ref)
    print('total:',len(test_data))
    print('bleu:',bleu)


In [ ]:
test()

INPUT ['When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', 'Envy', '.', '&quot;']
REF ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'Tôi', 'nghĩ', 'rằng', 'BắcTriều', 'Tiên', 'là', 'đất', 'nước', 'tốt', 'nhất', 'trên', 'thế', 'giới', 'và', 'tôi', 'thường', 'hát', 'bài', '&quot;', 'Chúng', 'ta', 'chẳng', 'có', 'gì', 'phải', 'ghen', 'tị', '.', '&quot;']
MT ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'tôi', 'nghĩ', 'đó', 'là', 'một', 'đất', 'nước', 'tuyệt', 'vời', 'tên', 'là']
INPUT ['And', 'I', 'was', 'very', 'proud', '.']
REF ['Tôi', 'đã', 'rất', 'tự', 'hào', 'về', 'đất', 'nước', 'tôi', '.']
MT ['Và', 'tôi', 'rất', 'tự', 'hào', '.']
INPUT ['In', 'school', ',', 'we', 'spent', 'a', 'lot', 'of', 'time', 'studying', 'the', 'history', 'of', 'Kim', 'Il-Sung', ',', 'but', 'we', 'never', 'learned', 'much', 'about', 'the', 'outside', 'world', ',', 'except', 'that', '